1. 거래 관련 데이터 (Transaction Data) 활용 전략

거래량과 네트워크 내 송금액을 분석하여 시장 참가자들의 활동 증가 또는 감소를 감지할 수 있습니다.

활성 주소 수 및 거래량 기반 트렌드 분석

지표: n-unique-addresses, n-transactions, transaction-volume

*blockchain.com에서 transaction-volume 지원 안해서 estimated-transaction-volume-usd로 대체함(해도 되는지는 모름)


전략:

활성 주소 및 거래량이 급격히 증가하면 강세 신호로 해석 (새로운 투자자 유입 가능성).

반대로, 활성 주소 및 거래량이 감소하면 약세 신호로 해석 (시장 참여 감소).

거래량 대비 가격 변화 분석: 거래량 증가에도 가격이 오르지 않으면 세력의 분배 가능성.

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_transaction_data(metric, start_date, end_date):
    """Blockchain Data API에서 거래 관련 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 2년 데이터 요청
start_date = datetime.now() - timedelta(days=730)
end_date = datetime.now()

# 가져올 거래 관련 데이터
transaction_metrics = ["n-unique-addresses", "n-transactions", "estimated-transaction-volume-usd"]
df_transaction = fetch_transaction_data(transaction_metrics[0], start_date, end_date)

for metric in transaction_metrics[1:]:
    temp_df = fetch_transaction_data(metric, start_date, end_date)
    if not temp_df.empty:
        df_transaction = df_transaction.merge(temp_df, on="timestamp", how="left")

# 데이터 저장
df_transaction.to_csv("bitcoin_transaction_data.csv", index=False)
print("✅ 거래 관련 데이터 저장 완료: bitcoin_transaction_data.csv")

2. 네트워크 보안 및 해시레이트 (Network & Mining Data) 활용 전략

비트코인 네트워크의 보안 상태 및 채굴자들의 행동을 분석하여 시장 안정성과 가격 변화를 예측할 수 있습니다.

-> 해시레이트와 난이도 기반 트렌드 분석

지표: hash-rate, difficulty

전략:

해시레이트 상승 → 네트워크 강세 (채굴자들이 장기 보유 예상) → 강세 신호

해시레이트 급락 → 채굴자들이 BTC를 덤핑할 가능성 → 약세 신호


📌 예시: 2021년 중국의 채굴 금지 조치 이후 해시레이트 급락 → BTC 가격 하락.

In [ ]:
def fetch_network_data(metric, start_date, end_date):
    """Blockchain Data API에서 네트워크 및 채굴 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 가져올 네트워크 관련 데이터
network_metrics = ["hash-rate", "difficulty"]
df_network = fetch_network_data(network_metrics[0], start_date, end_date)

for metric in network_metrics[1:]:
    temp_df = fetch_network_data(metric, start_date, end_date)
    if not temp_df.empty:
        df_network = df_network.merge(temp_df, on="timestamp", how="left")

# 데이터 저장
df_network.to_csv("bitcoin_network_data.csv", index=False)
print("✅ 네트워크 관련 데이터 저장 완료: bitcoin_network_data.csv")

3. UTXO 및 시장 활동 (UTXO & Market Activity) 활용 전략

UTXO는 비트코인 보유자들의 행동 패턴을 분석하는 데 중요한 역할을 합니다.

장기 보유자 vs 단기 보유자 비율 분석

지표: UTXO-age-distribution, LTH/STH supply ratio (얘네도 없어서 utxo 총 개수와 비트코인 총 거래량으로 대체.)

*아래 코드 utxo 총 개수만 읽음.. but 아래에 유동성 데이터에서 비트코인 총 거래량 읽었으므로 무시함*
***
전략:

장기 보유자(LTH, Long-Term Holders)의 BTC 보유량이 증가하면 강세 신호

단기 보유자(STH, Short-Term Holders)의 매도 증가 시 약세 신호

📌 예시: 2022년 BTC 하락장에서 장기 보유자들이 꾸준히 BTC를 축적한 후 2023년 반등.

**지표를 API 구조상 변경했으므로 이 코드가 이런 의미를 실제로 지닐 수 있는지는 추후 확인 필요**

In [ ]:
def fetch_utxo_data(metric, start_date, end_date):
    """Blockchain Data API에서 UTXO 관련 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 가져올 UTXO 관련 데이터
utxo_metrics = ["utxo-count", "total-bitcoins"]
df_utxo = fetch_utxo_data(utxo_metrics[0], start_date, end_date)

# 데이터 저장
df_utxo.to_csv("bitcoin_utxo_data.csv", index=False)
print("✅ UTXO 관련 데이터 저장 완료: bitcoin_utxo_data.csv")

4. 시장 심리 및 유동성 (Market Sentiment & Liquidity) 활용 전략

온체인 데이터를 활용해 시장 심리와 유동성을 분석할 수 있습니다.

원래 거래소 유입, 유출량 사용해야 하지만 API에 없음

유통중인 총 BTC 공급량과 시장가격으로 대체

In [ ]:
#유통중인 총 BTC 공급량
def fetch_exchange_flow_data(metric, start_date, end_date):
    """Blockchain Data API에서 거래소 유동성 관련 데이터를 가져오는 함수"""
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json&sampled=false"
    response = requests.get(url)
    data = response.json()

    if "values" not in data:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return pd.DataFrame()
    
    timestamps = [entry["x"] for entry in data["values"]]
    values = [entry["y"] for entry in data["values"]]

    df = pd.DataFrame({"timestamp": timestamps, metric: values})
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

    # 날짜 범위 필터링
    df = df[(df["timestamp"] >= start_date) & (df["timestamp"] <= end_date)]
    
    return df

# 가져올 거래소 유입/유출량 데이터
exchange_metrics = ["market-price", "total-bitcoins"]
df_exchange = fetch_exchange_flow_data(exchange_metrics[1], start_date, end_date)

# for metric in exchange_metrics[1:]:
#     temp_df = fetch_exchange_flow_data(metric, start_date, end_date)
#     if not temp_df.empty:
#         df_exchange = df_exchange.merge(temp_df, on="timestamp", how="left")

# 데이터 저장
df_exchange.to_csv("bitcoin_exchange_flow_data_2.csv", index=False)
print("✅ 거래소 유동성 데이터 저장 완료: bitcoin_exchange_flow_data_2.csv")

In [ ]:
#시장가격
import pandas as pd
import requests

# API 요청 함수
def get_bitcoin_data(metric):
    url = f"https://api.blockchain.info/charts/{metric}?timespan=2years&format=json"
    response = requests.get(url)
    data = response.json()
    if "values" in data:
        df = pd.DataFrame(data["values"])
        df["timestamp"] = pd.to_datetime(df["x"], unit="s").dt.strftime("%Y-%m-%d")
        df = df.rename(columns={"y": metric}).drop(columns=["x"])
        return df
    else:
        print(f"⚠️ API 응답 오류: '{metric}' 데이터에 'values' 키가 없습니다.")
        return None

# 데이터 가져오기
market_price_df = get_bitcoin_data("market-price")
total_bitcoins_df = get_bitcoin_data("total-bitcoins")

# 데이터 병합 (outer join)
df = pd.merge(market_price_df, total_bitcoins_df, on="timestamp", how="outer")

# CSV 저장
df.to_csv("bitcoin_exchange_flow_data.csv", index=False)